# Import Data

In [19]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve


In [33]:
x_train=pd.read_csv("/Users/xinyuewang/Desktop/Electricity Price Predict/X_train_NHkHMNU.csv")
y_train=pd.read_csv("/Users/xinyuewang/Desktop/Electricity Price Predict/y_train_ZAN5mwg.csv")
x_test=pd.read_csv("/Users/xinyuewang/Desktop/Electricity Price Predict/X_test_final.csv")
y_test=pd.read_csv("/Users/xinyuewang/Desktop/Electricity Price Predict/y_test_random_final.csv")

In [34]:
x_train.head()


,ID,DAY_ID,COUNTRY,DE_CONSUMPTION,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,DE_NET_EXPORT,FR_NET_EXPORT,DE_NET_IMPORT,...,FR_RESIDUAL_LOAD,DE_RAIN,FR_RAIN,DE_WIND,FR_WIND,DE_TEMP,FR_TEMP,GAS_RET,COAL_RET,CARBON_RET
0,1054,206,FR,0.210099,-0.427458,-0.606523,0.606523,NaN,0.692860,NaN,...,-0.444661,-0.172680,-0.556356,-0.790823,-0.283160,-1.069070,-0.063404,0.339041,0.124552,-0.002445
1,2049,501,FR,-0.022399,-1.003452,-0.022063,0.022063,-0.573520,-1.130838,0.573520,...,-1.183194,-1.240300,-0.770457,1.522331,0.828412,0.437419,1.831241,-0.659091,0.047114,-0.490365
2,1924,687,FR,1.395035,1.978665,1.021305,-1.021305,-0.622021,-1.682587,0.622021,...,1.947273,-0.480700,-0.313338,0.431134,0.487608,0.684884,0.114836,0.535974,0.743338,0.204952
3,297,720,DE,-0.983324,-0.849198,-0.839586,0.839586,-0.270870,0.563230,0.270870,...,-0.976974,-1.114838,-0.507570,-0.499409,-0.236249,0.350938,-0.417514,0.911652,-0.296168,1.073948
4,1101,818,FR,0.143807,-0.617038,-0.924990,0.924990,NaN,0.990324,NaN,...,-0.526267,-0.541465,-0.424550,-1.088158,-1.011560,0.614338,0.729495,0.245109,1.526606,2.614378


In [35]:
y_train.head()

,ID,TARGET
0,1054,0.028313
1,2049,-0.112516
2,1924,-0.180840
3,297,-0.260356
4,1101,-0.071733


# Data Preparation
There are some missing values in the dataset, so here we use the linear interpolation method to fill the missing value.

In [61]:
x_train.interpolate(method='polynomial',order=3, inplace=True)
x_train_clean = x_train.drop(['COUNTRY'], axis=1).fillna(0)
x_test.interpolate(method='polynomial',order=3, inplace=True)
x_test_clean = x_test.drop(['COUNTRY'], axis=1).fillna(0)

y_train_clean = y_train['TARGET']
y_test_clean=y_test['TARGET']


missing_values_x_train = x_train_clean.isnull().sum()
missing_values_x_train

missing_values_x_train = x_train.isnull().sum()
missing_values_x_train

print("\nDataFrame Info:\n", x_train.info())

shape = x_train.shape


print("DataFrame Shape:", shape)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1494 entries, 0 to 1493
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                1494 non-null   int64  
 1   DAY_ID            1494 non-null   int64  
 2   COUNTRY           1494 non-null   object 
 3   DE_CONSUMPTION    1494 non-null   float64
 4   FR_CONSUMPTION    1494 non-null   float64
 5   DE_FR_EXCHANGE    1494 non-null   float64
 6   FR_DE_EXCHANGE    1494 non-null   float64
 7   DE_NET_EXPORT     1493 non-null   float64
 8   FR_NET_EXPORT     1494 non-null   float64
 9   DE_NET_IMPORT     1493 non-null   float64
 10  FR_NET_IMPORT     1494 non-null   float64
 11  DE_GAS            1494 non-null   float64
 12  FR_GAS            1494 non-null   float64
 13  DE_COAL           1494 non-null   float64
 14  FR_COAL           1494 non-null   float64
 15  DE_HYDRO          1494 non-null   float64
 16  FR_HYDRO          1494 non-null   float64


# Random Forest 

In this part, we will use the random forest method to predict electricity price. We will follow a simple workflow as CART: tune the hyperparameters and then train the final model.

Random forest has two groups of hyperparameters:

1. The hyperparameters of decision trees, such as max_depth, min_samples_split;
2. The configurations of RF, such as the number of trees (called **n_estimators**). By default, **n_estimators** is set as 100. Some intuition is that the larger n_estimators, the more stable the model performance and the longer computing time. This is because the computing time is a linear function of the number of trees.

The default value of 100 trees is good for most applications, so we don't tune this hyperparameter here.

As with CART, we will focus on the tuning of max_depth and min_samples_split.


In [62]:
# values of max_depth and min_samples_split
hyperparameters = {'max_depth':[10,20,30,40,50], 'min_samples_split':[2,4,6,8,10]}

randomState_dt = 100
rf = RandomForestRegressor(random_state=randomState_dt)

#cv=5 by default, which means 5-fold cross-validation
#clf = GridSearchCV(rf, hyperparameters)

clf.fit(x_train_clean, y_train_clean)

# we can query the best parameter value and its accuracy score
print ("The best parameter value is: ")
print (clf.best_params_)
print ("The best score is: ")
print (clf.best_score_)

rf_final = RandomForestRegressor(max_depth=clf.best_params_['max_depth'], min_samples_split=clf.best_params_['min_samples_split'], random_state=randomState_dt)
rf_final.fit(x_train_clean, y_train_clean)

output_train = rf_final.predict(x_train_clean)
output_train

def metric_train(output):

    return  spearmanr(output, y_train_clean).correlation

print('Spearman correlation for the train set: {:.1f}%'.format(100 * metric_train(output_train) ))

The best parameter value is: 
{'max_depth': 10, 'min_samples_split': 8}
The best score is: 
-0.05392866999245436
Spearman correlation for the train set: 51.8%


In [63]:
y_test_submission = x_test[['ID']].copy()
y_test_submission['TARGET'] = rf_final.predict(x_test_clean)
y_test_submission.to_csv('benchmark_qrt.csv', index=False)
